In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
import re
import string
import mlflow
import dagshub
import pandas as pd
import numpy as np
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [21]:
base_dir = Path(os.getenv('BASE_DIR'))
base_data_dir = Path(os.getenv('BASE_DATA_DIR'))
external_data_path = os.path.join(base_data_dir,"external")
raw_data_path = os.path.join(base_data_dir, "raw")
notebooks_path = os.path.join(base_dir, "notebooks")
os.makedirs(raw_data_path, exist_ok=True)
os.makedirs(external_data_path, exist_ok=True)

In [10]:
df = pd.read_csv(os.path.join(external_data_path, 'tweet_emotions.csv')).drop(columns=['tweet_id'])
df.head(5)

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [12]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [13]:
df = normalize_text(df)
df.head(5)

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [14]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [16]:
x = df['sentiment'].isin(['sadness', 'happiness'])
df = df[x]
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\erdrr\AppData\Local\Temp\ipykernel_19116\2264095480.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want he s mar...
8,0,charviray charlene love miss
9,0,kelcouch i m sorry least friday


#### Apply BoW

In [17]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
mlflow.set_tracking_uri('https://dagshub.com/insculptor/mlops-mini-project.mlflow')
dagshub.init(repo_owner='insculptor', repo_name='mlops-mini-project', mlflow=True)

mlflow.set_experiment("mlops-mini-project_LR_baseline")

Accessing as insculptor

Initialized MLflow to track repo "insculptor/mlops-mini-project"

Repository insculptor/mlops-mini-project initialized!

2024/08/11 09:51:29 INFO mlflow.tracking.fluent: Experiment with name 'mlops-mini-project_LR_baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/c3b4d935be3d4e6ab31614989cf1cfaf', creation_time=1723384292404, experiment_id='1', last_update_time=1723384292404, lifecycle_stage='active', name='mlops-mini-project_LR_baseline', tags={}>

In [28]:
 with mlflow.start_run() as run:
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param('max_features', 1000)
    mlflow.log_param('test_size', 0.2)
    mlflow.set_tag("mlflow.runName", f"LR_baseline_{run.info.run_name}")
    
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    
    # log model paraemters
    mlflow.log_param('model', 'Logistic Regression')
    
    # MOdel evaluation
    y_pred = lr.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # log model metrics  
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1', f1)
     

    mlflow.sklearn.log_model(lr, "'model")
    
    ## Log Jupyter Notebook
    notebook_path = os.path.join(notebooks_path, "exp1_baseline_model.ipynb")
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1: {f1}")

c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/11 10:53:52 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 10:53:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR_baseline_upset-eel-849 at: https://dagshub.co

Accuracy: 0.779277108433735
Precision: 0.7701260911736179
Recall: 0.7822660098522167
F1: 0.7761485826001955
